In [216]:
# !pip install xlrd
# !pip install xlutils

In [236]:
import os
import json
import xlrd
import xlwt
import fiona
import shutil
import zipfile
import numpy as np
import pandas as pd
import geopandas as gpd

from glob import glob
from pprint import pprint
from xlutils.copy import copy
from pyproj import Proj, transform 

In [238]:
def sort_coordinates_clockwise(coords):
    """
    Sorts a set of 2D coordinates in clockwise order based on their angles from the centroid.

    Parameters:
    coords (list of tuples or numpy array): 2D points (x, y).

    Returns:
    list: Coordinates sorted in clockwise order.
    """
    # Calculate the centroid of the polygon
    centroid = np.mean(coords, axis=0)

    # Define an inner function to calculate the angle of each point with respect to the centroid
    def angle_from_centroid(point):
        """
        Calculates the angle of a point relative to the centroid using arctan2.

        Parameters:
        point (tuple): A point (x, y).

        Returns:
        float: The angle in radians.
        """
        return np.arctan2(point[1] - centroid[1], point[0] - centroid[0])

    # Sort the points by the calculated angles in descending order (clockwise)
    sorted_coords = sorted(coords, key=angle_from_centroid, reverse=True)
    return sorted_coords


def convert_to_utm(coords, zone='17S'):
    """
    Converts geographic coordinates (longitude, latitude) to UTM (Universal Transverse Mercator) coordinates.

    Parameters:
    coords (list of tuples): Geographic coordinates (longitude, latitude).
    zone (str): UTM zone, default is '17S'.

    Returns:
    list: UTM coordinates as (easting, northing).
    """
    # Define the WGS84 and UTM projections
    wgs84 = Proj(proj='latlong', datum='WGS84')  # WGS84 latitude/longitude projection
    utm = Proj(proj='utm', zone=17, south=True, datum='WGS84')  # UTM projection for the given zone

    utm_coords = []

    for lon, lat in coords:
        # Transform each coordinate from WGS84 to UTM
        easting, northing = transform(wgs84, utm, lon, lat)
        utm_coords.append((easting, northing))

    return utm_coords


def read_kml(file_path):
    """
    Reads a KML or KMZ file and converts its data to a GeoDataFrame.

    Parameters:
    file_path (str): Path to the .kml or .kmz file.

    Returns:
    tuple: A GeoDataFrame of the extracted features and a list of raw features.
    """
    # Enable Fiona's support for KML format
    fiona.drvsupport.supported_drivers['KML'] = 'rw'

    # Unzip the KML file if it's a KMZ (compressed)
    if file_path.endswith('.kmz'):
        with zipfile.ZipFile(file_path, 'r') as kmz:
            kmz.extractall('temp_kml')  # Extract to a temporary directory
        file_path = glob('temp_kml/*.kml')[0]  # Find the first extracted KML file

    # Read the KML file using Fiona
    with fiona.open(file_path, driver='KML') as src:
        features = list(src)  # Load features as a list

    # Convert the features to a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(features)

    # Clean up temporary files if any were created
    if os.path.exists('temp_kml'):
        shutil.rmtree('temp_kml')  # Remove the temporary directory

    return gdf, features


def crop_float(number, decimals):
    """
    Truncates a floating-point number to a specific number of decimal places without rounding.

    Parameters:
    number (float): The floating-point number to be truncated.
    decimals (int): The number of decimal places to retain.

    Returns:
    float: The truncated floating-point number.
    """
    # Calculate the truncation factor
    factor = 10 ** decimals

    # Truncate the number by scaling, converting to an integer, and scaling back down
    return int(number * factor) / factor

In [ ]:
dp = pd.read_csv("producers.txt", sep="\t")
dp

In [ ]:
dt = pd.read_csv("plots.txt", sep="\t")
dt.columns = ["L", "LL", "farm_id"]
dt

In [ ]:
merged_df = pd.merge(dt, dp, on='farm_id', how='inner')
merged_df["L"] = merged_df["L"].str.replace("ZN", "Z")
merged_df["LL"] = merged_df["LL"].str.replace("ZN", "Z") + ".kml"

nf = merged_df[merged_df.L.str.startswith("NF")]
nf

In [ ]:
# Group by column 'L' and concatenate the string values
grouped_df = nf.groupby('farm_id').agg(lambda x: ','.join(x.unique())).drop(columns=["L"])

grouped_df.reset_index(inplace=True)
grouped_df

In [ ]:
coords_folder = "nf"
os.makedirs(coords_folder, exist_ok=True)

file_path = 'Coordenadas área(s) geográfica(s).xls'
workbook = xlrd.open_workbook(file_path, formatting_info=True)
sheet = workbook.sheet_by_index(0)  # Use the first sheet

Q = {}
errores = []
c_kmls = 0
c_kmls2 = 0

for xx, row in grouped_df.iterrows():
    kml_code = [i.split("-")[0] for i in row["LL"].split(",")][0]
    kmls = sorted(glob(f"G4/*{kml_code}*kml"))
    c_kmls2 += len(kmls)
    producer = row["producer_name"]
    farm_id = row["farm_id"]

    # Make a writable copy of the workbook
    workbook_copy = copy(workbook)
    sheet_copy = workbook_copy.get_sheet(0)  # Select the first sheet to modify
    style_red = xlwt.easyxf("font: color red;")
    
    cont = 1
    cont3 = 1

    for ii, kml in enumerate(kmls):
        c_kmls += 1
        kml_n = kml.replace("-0", "-L")
        print(kml_n)
        gdf, features = read_kml(kml_n)
        coords = [(crop_float(i[0], 6), crop_float(i[1], 6)) for i in features[0].geometry.coordinates[0]]
        # Step 1: Sort the coordinates in clockwise order
        sorted_coords = sort_coordinates_clockwise(coords)
        # Step 2: Convert the sorted coordinates to UTM WGS84 Zone 17S
        utm_coords = convert_to_utm(sorted_coords)
        # utm_coords
        if producer not in Q:Q[producer] = [utm_coords]
        else:Q[producer] += [utm_coords]

        cont2 = 1
        for jj, (x, y) in enumerate(utm_coords):
            if jj==0:
                print(ii+1, cont3, x, y)
                sheet_copy.write(cont3, 0, ii+1)
                sheet_copy.write(cont3, 1, cont2)
                sheet_copy.write(cont3, 2, x, style_red)
                sheet_copy.write(cont3, 3, y, style_red)
                cont2+=1
                cont3+=1
            else:
                print(ii+1, cont3, x, y)
                sheet_copy.write(cont3, 0, ii+1)
                sheet_copy.write(cont3, 1, cont2)
                sheet_copy.write(cont3, 2, x)
                sheet_copy.write(cont3, 3, y)
                cont2+=1
                cont3+=1
        
        cont+=1

        print(ii+1, cont3, utm_coords[0][0], utm_coords[0][1])
        sheet_copy.write(cont3, 0, ii+1)
        sheet_copy.write(cont3, 1, cont2)
        sheet_copy.write(cont3, 2, utm_coords[0][0], style_red)
        sheet_copy.write(cont3, 3, utm_coords[0][1], style_red)

        cont3+=1

    # Save the modified workbook to a new file
    modified_file_path = f'{coords_folder}/Coordenadas_area_geografica_{farm_id}_{producer}.xls'
    workbook_copy.save(modified_file_path)
    
    del workbook_copy
    
    print()